In [ ]:
# imperative API

import gradelib

gb = gradelib.combine_gradebooks([
    gradelib.io.gradescope.read("./gradescope.csv"),
    gradelib.io.canvas.read("./canvas.csv")
])

homeworks = LazyAssignments(lamba asmts: asmts.starting_with('home'))
labs = LazyAssignments(lamba asmts: asmts.starting_with('labs'))

# 1. Assignment preprocessing

gb.combine_assignment_parts(gb, lambda s: s.split('-')[0], within=homeworks)
gb.combine_assignment_versions(gb, lambda s: s.split('0')[0], within=labs)
policies.redemption(gb, ['midterm 01', 'midterm 02 - redemption'])

# 2. Setting of assignment groups

gb.assignment_groups = {
    'homeworks': (gb.assignments.starting_with('home'), 0.25),
    'labs': (gb.assignments.starting_with('lab'), 0.25),
    'midterm 01 with redemption': 0.25,
    'midterm 02 with redemption': 0.25
}

# 3. Policies

policies.penalize_lates(gb, within=HOMEWORKS + LABS)
policies.drop_lowest(gb, within=HOMEWORKS)

In [ ]:
# steps API

import gradelib


gb = gradelib.combine_gradebooks([
    gradelib.io.gradescope.read("./gradescope.csv"),
    gradelib.io.canvas.read("./canvas.csv")
])


homeworks = LazyAssignments(lamba asmts: asmts.starting_with('home'))
labs = LazyAssignments(lamba asmts: asmts.starting_with('labs'))

# step 1
def preprocessing(gb):
    pre.combine_assignment_parts(lambda s: s.split('-')[0], within=homeworks)
    pre.combine_assignment_versions(lambda s: s.split('0')[0], within=labs)
    policies.redemption(gb, ['midterm 01', 'midterm 02 - redemption'])

    gb.assignment_groups = {
        'homeworks': (gb.assignments.starting_with('home'), 0.25),
        'labs': (gb.assignments.starting_with('lab'), 0.25),
        'midterm 01 with redemption': 0.25,
        'midterm 02 with redemption': 0.25
    }
    
# step 2    
def exceptions(gb):
    exceptions.make_exceptions({
        "justin": [
            Drop('homework 01', reason='sick'),
            ForgiveLate('homework 02', reason='sick'),
            Replace('homework 03', with_='homework 04', reason='late')
        ],
        
        "tyler": [
            Replace('homework 03', with_=Points(0), reason='Academic integrity violation')
        ]
    })

# step 3    
def policies(gb):
    policies.penalize_lates(gb, within=HOMEWORKS + LABS)
    policies.drop_lowest(gb, within=HOMEWORKS)
    
    
fgb = gb.apply([preprocessing, exceptions, policies])

In [ ]:
# declarative API

gb = gradelib.combine_gradebooks([
    gradelib.io.gradescope.read("./gradescope.csv"),
    gradelib.io.canvas.read("./canvas.csv")
])

homeworks = LazyAssignments(lamba asmts: asmts.starting_with('home'))
labs = LazyAssignments(lamba asmts: asmts.starting_with('labs'))

PREPROCESSING = [
    
    CombineAssignmentParts(
        lambda s: s.split()[0],
        within=homeworks
    ),
    
    CombineAssignmentVersions(
        lambda s: s.split()[0],
        within=labs
    ),
    
    Redemption({'midterm 01 with redemption': ['midterm 01', 'midterm 01 - redemption']}),
    
    GroupAssignments({
        'homeworks': (gb.assignments.starting_with('home'), 0.25),
        'labs': (gb.assignments.starting_with('lab'), 0.25),
        'midterm 01 with redemption': 0.25,
        'midterm 02 with redemption': 0.25
    }),
]

EXCEPTIONS = [
    Exception(
        "Justin", [
            Drop("homework 02", reason="Wasn't feeling well."),
            ForgiveLate("homework 01", reason="Sick."),
            
        ]
    ),
    
    Exception
]

POLICIES = [
    DropLowest(3, within=homeworks),
    PenalizeLates(forgive=7, within=homeworks)
]

fgb = gb.apply(PREPROCESSING + EXCEPTIONS + POLICIES)

In [ ]:
import gradelib
import gradelib.policies as gs

gb = gradelib.combine_gradebooks([
    gradelib.io.gradescope.read("./gradescope.csv"),
    gradelib.io.canvas.read("./canvas.csv")
])

gb = gb.apply([
    # merge homeworks and homework autograded parts
    gs.CombineAssignments(lambda s: s.split('-')[0]),
])

HOMEWORKS = LazyAssignments(lambda asmts: asmts.starting_with("homework"))
LABS = LazyAssignments(lambda asmts: asmts.starting_with("lab"))

EXCEPTIONS = [
    gs.Exception(
        "Justin Eldridge",
        forgive_lates=["homework 01", "homework 03"],
        drop=["homework 04"],
        replace=[("homework 05", "midterm 01")],
        adjust=[
            ("homework 06", Points(32)),
            ("midterm 02", Percentage(.85))
        ]
    )
]

POLICIES = [
    gs.PenalizeLates(
        forgive=3,
        within=lambda asmts: asmts.starting_with("homework"),
        penalty=Percentage(.75)
    ),
    
    gs.DropLowest(
        2,
        within=HOMEWORKS + LABS,
    ),
    
    gs.Redemption("midterm 01", "redemption midterm 01"),
    gs.Redemption("midterm 02", "redemption midterm 02") 
]

pipeline = [
    *EXCEPTIONS,
    *POLICIES
]

fgb = gb.grade(
    pipeline=pipeline,
)

fgb.opts.allow_extra_credit = True

fgb.groups = [
    (HOMEWORKS, 0.5),
    (LABS, 0.3),
    ('midterm 01', 0.25),
    ('midterm 02', 0.25),
    ('extra credit', .03)
]

fgb.scale = gradelib.io.read_scale()

scale = gradelib.scale.find_robust_scale(fgb)
gradelib.io.write_scale(scale, overwrite=True)

fgb.overall_scores
fgb.letter_grades
fgb.summarize()

fgb.summarize("someone")

In [ ]:
MutableGradebook

# these should all mutate the gradebook

.add_assignment()
.restrict_to_assignments()
.remove_assignment()
.combine_assignments()
.restrict_to_students()
.add_note()

# these should produce copies
.apply()
.finalize()
.grade() = .apply() + .finalize()

FinalizedGradebook